In [1]:
import pathlib
import textwrap
import os
import google.generativeai as genai
from IPython.display import display, Markdown, HTML
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import json
import re
import tiktoken

# Load environment variables
load_dotenv(find_dotenv())
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

# Configure the API client with the API key
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-pro')


In [2]:
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def upload_rubric(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    return content

# Documentation for Grading Functions

## Function: format_prompt

### Description
The `format_prompt` function constructs a prompt string that includes the grading rubric, the student's essay, and any additional information. This prompt is used to validate the rubric and grade the essay if the rubric is appropriate.

### Parameters
- **rubric_content** (`str`): The content of the grading rubric in string format.
- **essay_text** (`str`): The text of the essay to be graded.
- **additional_info** (`str`): Extra information to consider during grading.

### Returns
- **prompt** (`str`): A formatted string that includes the grading rubric, the student's essay, and the additional information, with specific instructions for validation and grading.

## Function: analyze_essay_with_rubric

### Description
The `analyze_essay_with_rubric` function takes a rubric content, an essay text, additional information, and a model to generate a grading feedback for the essay based on the rubric. The function constructs a prompt using the rubric content, essay text, and additional information, and then uses the provided AI model to generate the grading feedback.

### Parameters
- **model** (`GenerativeModel`): The AI model used to generate the content.
- **rubric_content** (`str`): The content of the grading rubric in string format.
- **essay_text** (`str`): The text of the essay to be graded.
- **additional_info** (`str`): Extra information to consider during grading.

### Returns
- **response.text** (`str`): The grading feedback generated by the model.

## Function: get_additional_info

### Description
The `get_additional_info` function allows the user to select or enter additional information that should be considered during grading. This additional information can influence the grading criteria and feedback provided by the model.

### Parameters
- None

### Returns
- **additional_info** (`str`): A string containing the selected or entered additional information.

### Usage
The function presents a list of preset options for additional information to the user. The user can choose one of these options or enter their own custom additional information. The selected or entered information is then returned as a string.

#### Preset Options:
1. Be extra harsh when grading
2. Add additional encouragement for the student
3. Focus on grammar and syntax
4. Consider creativity and originality
5. Suggest reading materials for the student based on the grading
6. Enter your own additional information


In [3]:
def format_prompt(rubric_content, essay_text, additional_info):
    return f"""
    Validate the following grading rubric for formatting and appropriateness:

    Grading Rubric:
    {rubric_content}

    Make careful judgement on the subject of the student essay. Is the rubric formatted correctly and appropriate for grading the essay? (yes or no) Please explain why for either case. 
    If no, notify the user that the rubric is not correponding to the student essay; do not grade the essay, but rather come up with a correct rubric and then the essay.
    If yes, grade the essay with the rubric.

    Student Response:
    {essay_text}

    Additional Information:
    {additional_info}
    
    If the rubric is originally correct, do NOT add new criterions to it.

    Sample Output Format:

    ### Grading Rubric and Feedback
    | Criterion | Score | Explanation |
    |-----------|-------|-------------|
    
    Please do not say criterion and the criterion number next to the actual criterion. Always provide an explanation.
    
    Calculate the final score by adding up the score for each criterion. If there is a weight, either in a new column or parantheses, please take weighted average.
    """

def analyze_essay_with_rubric(model, rubric_content, essay_text, additional_info):
    # Count input tokens for each component
    rubric_tokens = model.count_tokens(rubric_content).total_tokens
    essay_tokens = model.count_tokens(essay_text).total_tokens
    additional_info_tokens = model.count_tokens(additional_info).total_tokens

    print(f"Rubric Token Count: {rubric_tokens}")
    print(f"Essay Token Count: {essay_tokens}")
    print(f"Additional Info Token Count: {additional_info_tokens}")
    
    prompt = format_prompt(rubric_content, essay_text, additional_info)
    
    # Count input tokens for the entire prompt
    prompt_tokens = model.count_tokens(prompt).total_tokens
    print(f"Grading Prompt Token Count: {prompt_tokens}")
    
    response = model.generate_content(prompt)
    
    # Ensure the response streaming completes
    response.resolve()
    
    # Print token usage details
    usage_metadata = response.usage_metadata
    print(f"Grading Response Token Count: {usage_metadata.candidates_token_count}")
    print(f"Grading Total Token Count: {usage_metadata.total_token_count}")
    print(" ")

    
    return response.text


def get_additional_info():
    preset_options = [
        "Be extra harsh when grading",
        "Add additional encouragement for the student",
        "Focus on grammar and syntax",
        "Consider creativity and originality",
        "Suggest reading materials for the student based on the grading"
    ]
    
    print("Select additional information for grading:")
    for i, option in enumerate(preset_options, 1):
        print(f"{i}. {option}")
    print("6. Enter your own additional information")

    choice = int(input("Enter the number of your choice: "))

    if 1 <= choice <= 5:
        additional_info = preset_options[choice - 1]
    elif choice == 6:
        additional_info = input("Enter your additional information: ")
    else:
        print("Invalid choice, please select a valid option.")
        return get_additional_info()

    return additional_info

# Functions for Essay Grading and Improvement Suggestions

## Function: format_suggestions_and_rewrite_prompt

### Description
The `format_suggestions_and_rewrite_prompt` function constructs a prompt string that includes the grading feedback and the student's essay. This prompt is used to generate suggestions for improvement and to rewrite the essay using the suggested improvements.

### Parameters
- **grading_feedback** (`str`): The feedback received from grading the essay, which includes the criterion, score, and explanation.
- **essay_text** (`str`): The text of the essay to be improved and rewritten.

### Returns
- **prompt** (`str`): A formatted string that includes the grading feedback and the student's essay, with specific instructions for generating suggestions and rewriting the essay. The instructions ask for the smallest chunk of the original text where each improvement was applied, the criterion from the rubric, and the reason for the suggested improvement, all formatted in JSON.

## Function: generate_suggestions_and_rewrite_essay

### Description
The `generate_suggestions_and_rewrite_essay` function takes grading feedback, an essay text, and a model to generate suggestions for improvement and rewrite the essay using those suggestions. The function constructs a prompt using the `format_suggestions_and_rewrite_prompt` function, which combines the grading feedback and essay text into a single prompt. It then uses the provided AI model to generate the suggestions and rewritten essay.

### Parameters
- **model** (`GenerativeModel`): The AI model used to generate the content.
- **grading_feedback** (`str`): The feedback received from grading the essay, which includes the criterion, score, and explanation.
- **essay_text** (`str`): The text of the essay to be improved and rewritten.
- **retries** (`int`, optional): The number of times to retry the API request in case of resource exhaustion. Default is 3.

### Returns
- **response.text** (`str`): The suggestions and rewritten essay generated by the model. If the API request fails after the specified number of retries, the function returns `None`.

### How it Works
1. **Construct Prompt**: The function calls the `format_suggestions_and_rewrite_prompt` function, passing in the grading feedback and essay text. This function returns a formatted prompt string.
2. **Generate Content**: The function uses the AI model to generate content based on the constructed prompt. It attempts to call the model's `generate_content` method.
3. **Retry Mechanism**: If the API request fails due to resource exhaustion, the function catches the `ResourceExhausted` exception and retries the request up to the specified number of times, with exponential backoff between attempts.
4. **Return Response**: If the model successfully generates a response, the function returns the generated text. If the request fails after all retries, the function returns `None`.


In [4]:
def format_suggestions_and_rewrite_prompt(grading_feedback, essay_text):
    return f"""
    Based on the following grading feedback, 
    
    1. Provide suggestions for improvement
    2. Rewrite the essay using the suggested improvements

    Grading Feedback:
    {grading_feedback}

    Essay:
    {essay_text}
    
    Take each improvement provided and identify,
    a. the smallest chunk of the original text where it was applied
    b. the criterion from the rubric 
    c. the reason for the suggested improvement
    
    Provide steps a, b, and c in the following JSON format: 
    [
        {{"improvement": "improvement_1", "criterion_from_rubric": "criterion_from_rubric_1", "reason_for_suggestion": "reason_for_suggestion_1", "original_text": "original_text_1", "revised_text": "revised_text_1"}},
        ...
        {{"improvement": "improvement_n", "criterion_from_rubric": "criterion_from_rubric_n", "reason_for_suggestion": "reason_for_suggestion_n", "original_text": "original_text_n", "revised_text": "original_text_n"}}
    ]
    

    """

def generate_suggestions_and_rewrite_essay(model, grading_feedback, essay_text):
    # Count input tokens for each component
    feedback_tokens = model.count_tokens(grading_feedback).total_tokens
    essay_tokens = model.count_tokens(essay_text).total_tokens

    print(f"Grading Feedback Token Count: {feedback_tokens}")
    print(f"Essay Token Count: {essay_tokens}")
    
    prompt = format_suggestions_and_rewrite_prompt(grading_feedback, essay_text)
    
    # Count input tokens for the entire prompt
    prompt_tokens = model.count_tokens(prompt).total_tokens
    print(f"Suggestions Prompt Token Count: {prompt_tokens}")
    
    response = model.generate_content(prompt)
    
    # Ensure the response streaming completes
    response.resolve()
    
    # Print token usage details
    usage_metadata = response.usage_metadata
    print(f"Suggestions Response Token Count: {usage_metadata.candidates_token_count}")
    print(f"Suggestions Total Token Count: {usage_metadata.total_token_count}")
    print(" ")
    
    if response and response.text:
        print("Full response from model:\n", response.text)  # Debugging statement
        return response.text
    else:
        print("Error: No response from model")
        return None


# Functions to Clean the JSON Output

In [5]:
def clean_json_string(json_string):
    # Remove Markdown formatting and trailing commas
    json_string = json_string.replace('```json', '').replace('```', '').strip()
    json_string = re.sub(r',\s*([}\]])', r'\1', json_string)  # Remove trailing commas before closing braces
    # Ensure all objects are closed properly
    json_string = re.sub(r'\s*}\s*{', '},{', json_string)
    # Add missing closing brackets if necessary
    if json_string.count('[') > json_string.count(']'):
        json_string += ']'
    if json_string.count('{') > json_string.count('}'):
        json_string += '}'
    return json_string

def extract_json_improvements(response):
    if response is None:
        print("Error: No response provided for JSON extraction")
        return {'improvements': [], 'response_without_json': ''}
    try:
        json_match = re.search(r'\[\s*\{.*\}\s*\]', response, re.DOTALL)
        if not json_match:
            raise ValueError("No valid JSON found in response.")
        
        cleaned_json_string = clean_json_string(json_match.group(0))
        print("Cleaned JSON string:", cleaned_json_string)  # Debugging statement

        improvements = json.loads(cleaned_json_string)
        response_without_json = response.replace(json_match.group(0), '')
        return {'improvements': improvements, 'response_without_json': response_without_json}
    except (ValueError, json.JSONDecodeError) as e:
        print("Error extracting JSON from response:", e)
        print(f"Failed to decode JSON string: {response}")  # Print the entire response for debugging
        return {'improvements': [], 'response_without_json': response}


# Function to Display Improvements

In [6]:
def display_improvements(improvements, original_text):
    if not improvements:
        print("No improvements to display.")
        return

    css = """
    <style>
    .improvement-tooltip {
        position: relative;
        display: inline-block;
        cursor: pointer;
        background-color: yellow;
        margin-bottom: 10px;
        padding: 5px;
        border: 1px solid black;
        border-radius: 4px;
    }
    .improvement-tooltip .tooltiptext {
        visibility: hidden;
        width: 300px;
        background-color: white;
        color: black;
        text-align: left;
        border: 1px solid #ddd;
        padding: 10px;
        border-radius: 6px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
        position: absolute;
        z-index: 1;
        top: 100%;
        left: 50%;
        margin-left: -150px;
    }
    .improvement-tooltip:hover .tooltiptext {
        visibility: visible;
    }
    </style>
    """
    
    html_content = original_text
    
    for improvement in improvements:
        original = improvement.get('original_text', '')
        revised = improvement.get('revised_text', '')
        improvement_text = improvement['improvement']
        reason_for_suggestion = improvement['reason_for_suggestion']
        
        # Check if 'criterion_from_rubric' is present
        if 'criterion_from_rubric' in improvement:
            criterion = improvement['criterion_from_rubric']
            tooltip_html = f"""
            <div class="improvement-tooltip">
                {revised}
                <span class="tooltiptext">
                    <strong>Criterion:</strong> {criterion}<br>
                    <strong>Improvement:</strong> {improvement_text}<br>
                    <strong>Reason:</strong> {reason_for_suggestion}
                </span>
            </div>
            """
        else:
            tooltip_html = f"""
            <div class="improvement-tooltip">
                {revised}
                <span class="tooltiptext">
                    <strong>Improvement:</strong> {improvement_text}<br>
                    <strong>Reason:</strong> {reason_for_suggestion}
                </span>
            </div>
            """
        
        if original:
            html_content = html_content.replace(original, tooltip_html)
        else:
            html_content = html_content + tooltip_html
    
    display(HTML(css + html_content))


# Functions for Selecting and Listing Rubrics

In [7]:
def list_rubrics(rubric_folder):
    return [f for f in os.listdir(rubric_folder) if f.endswith('.md')]

def select_rubric(rubric_folder):
    rubrics = list_rubrics(rubric_folder)
    print("Available rubrics:")
    for idx, rubric in enumerate(rubrics, 1):
        print(f"{idx}. {rubric}")
    
    rubric_index = int(input("Select a rubric by entering the corresponding number: ")) - 1
    selected_rubric = rubrics[rubric_index]
    return os.path.join(rubric_folder, selected_rubric)

current_dir = os.getcwd()
relative_rubric_folder = '../rubrics'
rubric_folder = os.path.join(current_dir, relative_rubric_folder)

if not os.path.exists(rubric_folder):
    print(f"The directory {rubric_folder} does not exist. Creating the directory.")
    os.makedirs(rubric_folder)
    print(f"Please add rubric files in markdown format (.md) to the {rubric_folder} directory and re-run the notebook.")
else:
    rubric_files = list_rubrics(rubric_folder)
    if not rubric_files:
        print(f"No rubric files found in the directory {rubric_folder}. Please add rubric files in markdown format (.md) and re-run the notebook.")
    else:
        rubric_file_path = select_rubric(rubric_folder)
        if rubric_file_path:
            rubric_content = upload_rubric(rubric_file_path)
            display(to_markdown(rubric_content))

Available rubrics:
1. writing_ela_rubric_11_12_nys.md
2. writing_ela_rubric_9_10_nys.md
3. writing_social_sciences_11_12_nys.md
4. science_rubric_9_12_nys.md
Select a rubric by entering the corresponding number: 3


> 
> # New York State Common Core Writing Standards Rubric for Social Sciences (Grades 11-12)
> 
> | Criterion                  | 1 | 2 | 3 | 4 | 5 |
> |----------------------------|---|---|---|---|---|
> | **1. Argument Development** | Introduces claims with little or no clarity; minimal distinction between claims and counterclaims; lacks logical organization | Introduces claims with some clarity; some distinction between claims and counterclaims; basic organization but lacks depth | Introduces clear claims; distinguishes claims from counterclaims; logical sequence but may have minor gaps | Introduces precise and knowledgeable claims; clearly distinguishes claims from counterclaims; logically sequences with minor issues | Introduces precise, knowledgeable claims; clearly distinguishes and logically sequences claims, counterclaims, reasons, and evidence |
> | **2. Evidence and Analysis** | Provides minimal or irrelevant evidence; weak development of claims and counterclaims; lacks consideration of audience | Provides some relevant evidence; basic development of claims and counterclaims; limited audience consideration | Provides relevant evidence and analysis; adequately develops claims and counterclaims; considers audience to some extent | Provides strong evidence and analysis; thoroughly develops claims and counterclaims; anticipates audience’s knowledge and biases | Provides the most relevant evidence and analysis; thoroughly develops claims and counterclaims with strong audience consideration |
> | **3. Cohesion and Clarity** | Minimal use of transitions and varied syntax; weak cohesion and unclear relationships between ideas | Uses some transitions and varied syntax; some cohesion but relationships between ideas may be unclear | Uses appropriate transitions and varied syntax; clear relationships between most ideas | Uses varied syntax and transitions effectively; clear and cohesive relationships between ideas | Uses varied syntax and transitions skillfully; creates strong cohesion and clear relationships between all ideas |
> | **4. Style and Tone** | Inconsistent or inappropriate style and tone; minimal adherence to discipline norms | Somewhat consistent style and tone; some adherence to discipline norms | Consistent style and tone; adheres to discipline norms but may have minor lapses | Maintains formal style and objective tone; adheres to discipline norms consistently | Establishes and maintains formal style and objective tone; fully adheres to discipline norms |
> | **5. Conclusion** | Provides a weak or irrelevant conclusion; does not support the argument | Provides a basic conclusion; somewhat supports the argument | Provides a clear conclusion; supports the argument but may lack depth | Provides a strong conclusion; effectively supports the argument | Provides a compelling conclusion; thoroughly supports the argument and adds insight |
> 


In [8]:
# Get additional information from the user
additional_info = get_additional_info()

Select additional information for grading:
1. Be extra harsh when grading
2. Add additional encouragement for the student
3. Focus on grammar and syntax
4. Consider creativity and originality
5. Suggest reading materials for the student based on the grading
6. Enter your own additional information
Enter the number of your choice: 3


In [9]:
# The essay text
essay_text =  """
The history of the United States is marked by a continuous struggle for civil rights and equality, shaping the nation's identity and societal structure. From the early days of colonization to the modern era, the quest for civil rights has been a central theme, reflecting the dynamic and often tumultuous nature of the American democratic experiment. This ongoing journey has significantly influenced the development of American society and democracy.
The roots of the civil rights movement can be traced back to the early colonial period, where notions of freedom and equality were paradoxically juxtaposed with the institution of slavery. The Declaration of Independence in 1776, with its proclamation that "all men are created equal," laid a philosophical foundation for civil rights, yet this ideal was not extended to all individuals, particularly African Americans and Native Americans. The contradiction between the ideals of the Declaration and the reality of slavery set the stage for future conflicts and movements aimed at achieving true equality.
The abolitionist movement of the 19th century was one of the earliest organized efforts to challenge slavery and advocate for the civil rights of African Americans. Figures such as Frederick Douglass, Harriet Tubman, and William Lloyd Garrison played pivotal roles in raising awareness and mobilizing support for the abolition of slavery. The publication of antislavery literature, such as Harriet Beecher Stowe's Uncle Tom's Cabin, also contributed to the growing sentiment against slavery in the North.
The culmination of the abolitionist movement was the Civil War (1861-1865), a conflict fundamentally rooted in the issue of slavery. The Union's victory and the subsequent passage of the 13th Amendment in 1865 abolished slavery, marking a significant milestone in the struggle for civil rights. However, the end of slavery did not mean the end of discrimination or the beginning of true equality. The Reconstruction era (1865-1877) saw significant advancements, including the 14th and 15th Amendments, which granted citizenship and voting rights to African Americans. Despite these legal gains, the period was also marked by severe backlash, including the rise of the Ku Klux Klan and the implementation of Jim Crow laws, which enforced racial segregation and disenfranchised Black citizens.
The early 20th century witnessed the emergence of new civil rights organizations and leaders who sought to challenge segregation and discrimination through legal means and grassroots activism. The National Association for the Advancement of Colored People (NAACP), founded in 1909, played a crucial role in challenging discriminatory laws and practices through the court system. One of the most significant victories was the Supreme Court's decision in Brown v. Board of Education (1954), which declared state laws establishing separate public schools for Black and white students to be unconstitutional.
The 1950s and 1960s are often regarded as the height of the civil rights movement, a period characterized by widespread activism, landmark legislation, and profound social change. The movement was marked by nonviolent protests, such as the Montgomery Bus Boycott (1955-1956) and the March on Washington (1963), where Dr. Martin Luther King Jr. delivered his iconic "I Have a Dream" speech. These actions, along with the courage and determination of countless individuals, brought national attention to the injustices faced by African Americans.
Key legislative achievements during this period included the Civil Rights Act of 1964 and the Voting Rights Act of 1965. The Civil Rights Act prohibited discrimination on the basis of race, color, religion, sex, or national origin and ended segregation in public places. The Voting Rights Act aimed to eliminate barriers to voting for African Americans, particularly in the South, where practices such as literacy tests and poll taxes had been used to disenfranchise Black voters.
Despite these significant advancements, the struggle for civil rights continued beyond the 1960s. The late 20th and early 21st centuries saw ongoing efforts to address issues such as racial profiling, police brutality, and economic inequality. The Black Lives Matter movement, which emerged in response to the killings of unarmed Black individuals by police, has brought renewed attention to the persistent racial disparities in American society.
The quest for civil rights in the United States has not been limited to African Americans. Other marginalized groups, including women, Native Americans, LGBTQ+ individuals, and immigrants, have also fought for recognition and equality. The women's suffrage movement, which culminated in the 19th Amendment in 1920, granted women the right to vote and marked a significant step toward gender equality. The LGBTQ+ rights movement achieved a major victory with the Supreme Court's decision in Obergefell v. Hodges (2015), which legalized same-sex marriage nationwide.
Throughout American history, the struggle for civil rights has been a dynamic and multifaceted endeavor. It has involved legal battles, grassroots activism, and significant sacrifices by individuals committed to the cause of equality. The progress achieved has been substantial, yet the journey is ongoing, with new challenges and opportunities for advancing civil rights continuing to emerge.
In conclusion, the history of civil rights in the United States is a testament to the resilience and determination of those who have fought for justice and equality. From the abolition of slavery to the civil rights movement of the 1960s and beyond, each generation has contributed to the ongoing effort to realize the ideals of freedom and equality upon which the nation was founded. As society continues to evolve, the legacy of these struggles serves as both a reminder of the progress made and a call to action to address the inequalities that persist today. The journey toward equality is far from over, but the commitment to civil rights remains a defining aspect of the American spirit. Each new movement and each legal victory brings the nation closer to the ideal of true equality, demonstrating that the pursuit of civil rights is an enduring and essential part of the American story. The impact of these movements reverberates through history, showcasing the power of collective action and the relentless pursuit of justice. As new challenges arise, the foundational values of equality and justice continue to guide and inspire future generations to push forward.
"""


In [10]:
# Validate and grade the essay
grading_feedback = analyze_essay_with_rubric(model, rubric_content, essay_text, additional_info)
print("Grading Feedback:")
print(grading_feedback)
display(to_markdown(grading_feedback))


Rubric Token Count: 505
Essay Token Count: 1270
Additional Info Token Count: 5
Grading Prompt Token Count: 2018
Grading Response Token Count: 324
Grading Total Token Count: 2342
 
Grading Feedback:
### Grading Rubric and Feedback

| Criterion                  | Score | Explanation |
|----------------------------|-------|-------------|
| Argument Development       | 5     | The essay presents a clear and knowledgeable argument about the evolution of civil rights in the US. It logically sequences historical periods and events to demonstrate the ongoing struggle for equality.  |
| Evidence and Analysis      | 4     | The essay effectively uses historical examples like the Civil War, landmark legislation, and key figures to support its claims. However, a deeper analysis of specific events or figures could further strengthen the essay. |
| Cohesion and Clarity       | 5     | The essay demonstrates skillful use of varied syntax and transitions, creating a cohesive and clear flow of ideas. T

> ### Grading Rubric and Feedback
> 
> | Criterion                  | Score | Explanation |
> |----------------------------|-------|-------------|
> | Argument Development       | 5     | The essay presents a clear and knowledgeable argument about the evolution of civil rights in the US. It logically sequences historical periods and events to demonstrate the ongoing struggle for equality.  |
> | Evidence and Analysis      | 4     | The essay effectively uses historical examples like the Civil War, landmark legislation, and key figures to support its claims. However, a deeper analysis of specific events or figures could further strengthen the essay. |
> | Cohesion and Clarity       | 5     | The essay demonstrates skillful use of varied syntax and transitions, creating a cohesive and clear flow of ideas. The relationships between historical periods and events are well-explained. |
> | Style and Tone            | 4     | The essay maintains a formal style and objective tone appropriate for historical analysis. However, some sentences could be more concise for improved clarity.  |
> | Conclusion                 | 5     | The essay provides a compelling conclusion that summarizes the significance of the civil rights movement and its ongoing impact on American society. It effectively supports the argument and offers insightful reflections. |
> 
> **Final Score:** 23/25 
> 
> **Overall Feedback:** This is a well-written essay that demonstrates a strong understanding of the historical development of civil rights in the United States. The essay effectively uses historical evidence and presents a clear and well-organized argument. With some minor improvements in sentence-level clarity and depth of analysis, this essay could be even stronger. 


In [11]:
# Generate suggestions and rewrite the essay based on the grading feedback
suggestions_and_rewrite = generate_suggestions_and_rewrite_essay(model, grading_feedback, essay_text)
print(suggestions_and_rewrite)

Grading Feedback Token Count: 326
Essay Token Count: 1270
Suggestions Prompt Token Count: 1852
Suggestions Response Token Count: 1223
Suggestions Total Token Count: 3075
 
Full response from model:
 ```json
[
  {
    "improvement": "Condense the first sentence of the second paragraph for clarity.",
    "criterion_from_rubric": "Style and Tone",
    "reason_for_suggestion": "The original sentence is a bit lengthy and can be made more concise.",
    "original_text": "The roots of the civil rights movement can be traced back to the early colonial period, where notions of freedom and equality were paradoxically juxtaposed with the institution of slavery.",
    "revised_text": "The civil rights movement has roots in the colonial period, where ideals of freedom and equality clashed with the reality of slavery."
  },
  {
    "improvement": "Instead of just mentioning figures, delve deeper into their specific contributions to the abolitionist movement.",
    "criterion_from_rubric": "Evidence 

In [12]:
extracted_data = extract_json_improvements(suggestions_and_rewrite)
print("Extracted Data:", extracted_data)  # Debugging statement


Cleaned JSON string: [
  {
    "improvement": "Condense the first sentence of the second paragraph for clarity.",
    "criterion_from_rubric": "Style and Tone",
    "reason_for_suggestion": "The original sentence is a bit lengthy and can be made more concise.",
    "original_text": "The roots of the civil rights movement can be traced back to the early colonial period, where notions of freedom and equality were paradoxically juxtaposed with the institution of slavery.",
    "revised_text": "The civil rights movement has roots in the colonial period, where ideals of freedom and equality clashed with the reality of slavery."
  },
  {
    "improvement": "Instead of just mentioning figures, delve deeper into their specific contributions to the abolitionist movement.",
    "criterion_from_rubric": "Evidence and Analysis",
    "reason_for_suggestion": "Provides a more in-depth analysis of the historical figures mentioned and their impact.",
    "original_text": "Figures such as Frederick Dou

In [13]:
# Display the suggestions and revised essay
display(Markdown(f"**Suggestions and Revised Essay:**\n\n{extracted_data['response_without_json']}"))


**Suggestions and Revised Essay:**

```json

```


In [14]:
# Display the improvements in the new essay with hover effect
display_improvements(extracted_data['improvements'], essay_text)
